In [3]:
from selenium import webdriver
from urllib import parse
import bs4
import re
import time
import pandas as pd
def load_data(State):
    dblocation = '/Users/graceli/Desktop/SBR automation/Company Database.db'
    import pandas as pd
    import sqlite3
    conn = sqlite3.connect(dblocation)

    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    data = pd.read_sql_query("SELECT CompanyName from CompanyData WHERE State='"+State+"'", conn)
    return data['CompanyName']
def get_soup(driver):
    html = driver.page_source
    soup = bs4.BeautifulSoup(html,'html.parser')
    return soup
def save_result(results,State):
    results = pd.DataFrame(results)[['CompanyName','regName','regNumber','RegDate', 'Jurisdiction','entType','status','address']]
    results.to_csv(State+'_results.csv',index=False)
    return results


## Connect database

In [4]:
AR = load_data('AR') 

In [5]:
AR

0                                Advanced Environmental
1     Advanced Environmental Recycling Technologies,...
2                       Ainsworth Pet Nutrition (Dumas)
3                                Airtherm Products Inc.
4                                       American Tubing
5                       Armstrong Hardwood Flooring Co.
6                          Ashley County Medical Center
7                           Atlantis Plastics Injection
8                                Basic Construction Co.
9                               Baxter Healthcare Corp.
10                       Baxter Regional Medical Center
11                    Boozer Laminated Beam Company Inc
12                                        Bradleys' Inc
13                     Centinela Freeman Reg Med Center
14                                          Century LLC
15                          Child Craft Industries Inc.
16                                  Clyde Bergemann Ltd
17                                   Custom Prof

## Search function

In [11]:
def ARbusiness(business):
    url = 'https://www.sos.arkansas.gov/corps/search_all.php'
    driver = webdriver.Chrome()
    driver.get(url)

    time.sleep(1)
    search = driver.find_element_by_xpath('//*[@id="mainContent"]/form/table/tbody/tr[4]/td[2]/font/input')
    search.send_keys(business)


    submit = driver.find_element_by_xpath('//*[@id="mainContent"]/form/table/tbody/tr[11]/td/font/input')
    submit.click()


    if get_soup(driver).find(string='There were no records found!') != None:
        driver.quit()
        return [' ']*7

    driver\
        .find_element_by_xpath('//*[@id="mainContent"]/table[3]/tbody/tr[2]/td[1]/font/div/a')\
        .click()
 
    time.sleep(3)
    
    
    soup = get_soup(driver)
    company_l = list(map(lambda x: x.text.strip().split('\n') ,soup.findAll('tr')))
    company_l = [x for x in company_l if len(x) == 2]

    infodic = {'Principal Address':' ',
           'Filing #':' ',
           'Corporation Name': ' ',
           'Date Filed': ' ',
           'State of Origin': ' ',
           'Filing Type': ' ',
           'Status': ' '}    

    for attr in company_l:
        infodic[attr[0]] = attr[1]

    address = infodic['Principal Address']
    regNumber = infodic['Filing #']
    regName = infodic['Corporation Name']
    RegDate = infodic['Date Filed']
    Jurisdiction = infodic['State of Origin']
    entType =  infodic['Filing Type']
    status = infodic['Status']

    
    driver.quit()
    return [regNumber, regName, RegDate, Jurisdiction, entType, status, address]


## Save Results

In [14]:
AR_results = []

In [15]:
for business in AR[:100]:
    searchResult = ARbusiness(business)
    AR_results.append({'CompanyName':business,'regNumber':searchResult[0], 'regName':searchResult[1],
                                    'RegDate':searchResult[2], 'Jurisdiction':searchResult[3],
                                     'entType':searchResult[4],
                                     'status': searchResult[5],'address':searchResult[6]})

In [16]:
save_result(AR_results,'AR')

,CompanyName,regName,regNumber,RegDate,Jurisdiction,entType,status,address
0,Advanced Environmental,ADVANCED ENVIRONMENTAL & CONSTRUCTION LLC,811036341,07/03/2013,N/A,Limited Liability Company,Dissolved,"180 HASSEL WAYROSE BUD, AR 72137"
1,"Advanced Environmental Recycling Technologies,...","ADVANCED ENVIRONMENTAL RECYCLING TECHNOLOGIES,...",100076053,03/07/1989,DE,Foreign For Profit Corporation,Good Standing,
2,Ainsworth Pet Nutrition (Dumas),,,,,,,
3,Airtherm Products Inc.,,,,,,,
4,American Tubing,"AMERICAN TUBING ARKANSAS, LLC",811070600,02/19/2015,N/A,Name Reservation,Expired,
5,Armstrong Hardwood Flooring Co.,,,,,,,
6,Ashley County Medical Center,,,,,,,
7,Atlantis Plastics Injection,"ATLANTIS PLASTICS INJECTION MOLDING, INC.",100087054,12/10/1990,KY,Foreign For Profit Corporation,Revoked,
8,Basic Construction Co.,BASIC CONSTRUCTION CO.,100005682,03/17/1988,N/A,For Profit Corporation,Revoked,
9,Baxter Healthcare Corp.,,,,,,,
